Load data

In [1]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

dataPath = "C:/Users/Maevex/Desktop/Lujain/home-credit-credit-risk-model-stability/"

Reading the Train and Test Data

In [2]:
# Read CSV files
train_basetable = pd.read_csv(dataPath + "csv_files/train/train_base.csv")

train_static_0_0 = pd.read_csv(dataPath + "csv_files/train/train_static_0_0.csv")

train_static_0_1 = pd.read_csv(dataPath + "csv_files/train/train_static_0_1.csv")

# Concatenate the DataFrames vertically
train_static = pd.concat([train_static_0_0, train_static_0_1], axis=0, ignore_index=True)

train_static_cb = pd.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv")

train_person_1 = pd.read_csv(dataPath + "csv_files/train/train_person_1.csv")

train_credit_bureau_b_2 = pd.read_csv(dataPath + "csv_files/train/train_credit_bureau_b_2.csv")


C:\Users\Maevex\AppData\Local\Temp\ipykernel_20684\937885079.py:4: DtypeWarning: Columns (20,45,46,53,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_static_0_0 = pd.read_csv(dataPath + "csv_files/train/train_static_0_0.csv")
C:\Users\Maevex\AppData\Local\Temp\ipykernel_20684\937885079.py:6: DtypeWarning: Columns (20,45,46,56,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_static_0_1 = pd.read_csv(dataPath + "csv_files/train/train_static_0_1.csv")
C:\Users\Maevex\AppData\Local\Temp\ipykernel_20684\937885079.py:11: DtypeWarning: Columns (1,2,3,4,7,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  train_static_cb = pd.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv")
C:\Users\Maevex\AppData\Local\Temp\ipykernel_20684\937885079.py:13: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
 

In [3]:
# Read CSV files for test data
test_basetable = pd.read_csv(dataPath + "csv_files/test/test_base.csv")

test_static_0_0 = pd.read_csv(dataPath + "csv_files/test/test_static_0_0.csv")

test_static_0_1 = pd.read_csv(dataPath + "csv_files/test/test_static_0_1.csv")

test_static_0_2 = pd.read_csv(dataPath + "csv_files/test/test_static_0_2.csv")

# Concatenate the DataFrames vertically
test_static = pd.concat([test_static_0_0, test_static_0_1, test_static_0_2], axis=0, ignore_index=True)

test_static_cb = pd.read_csv(dataPath + "csv_files/test/test_static_cb_0.csv")

test_person_1 = pd.read_csv(dataPath + "csv_files/test/test_person_1.csv")

test_credit_bureau_b_2 = pd.read_csv(dataPath + "csv_files/test/test_credit_bureau_b_2.csv")


Finding nulls values

In [4]:
import pandas as pd

# Function to find null values and their percentages
def find_nulls(df):
    null_counts = df.isnull().sum()
    total_counts = len(df)
    null_percentages = (null_counts / total_counts) * 100
    return pd.DataFrame({
        'Null Count': null_counts,
        'Null Percentage': null_percentages
    })


# Finding nulls in each dataframe for the train data
train_nulls = {
    "train_basetable": find_nulls(train_basetable),
    "train_static": find_nulls(train_static),
    "train_static_cb": find_nulls(train_static_cb),
    "train_person_1": find_nulls(train_person_1),
    "train_credit_bureau_b_2": find_nulls(train_credit_bureau_b_2)
}

# Convert the dictionaries to DataFrames
train_nulls_df = pd.concat(train_nulls, axis=1)

# Exporting the nulls count to an Excel file
with pd.ExcelWriter(dataPath + 'null_counts.xlsx') as writer:
    train_nulls_df.to_excel(writer, sheet_name='train_nulls', index=True)


Casting types

In [5]:
import pandas as pd

def determine_dtype(col_name):
    if col_name[-1] == 'D':
        return 'datetime64[ns]'
    elif col_name[-1] == 'L':
        return 'object'
    elif col_name[-1] == 'A':
        return 'float64'
    elif col_name[-1] == 'M':
        return 'object'
    elif col_name[-1] == 'P':
        return 'float64'
    elif col_name[-1] == 'T':
        return 'object'
    else:
        return 'object'

def set_table_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        dtype = determine_dtype(col)
        df[col] = df[col].astype(dtype)
    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df




In [6]:
# Read CSV files
train_static_0_0 = set_table_dtypes(train_static_0_0)
train_static_0_1 = set_table_dtypes(train_static_0_1)
train_static_cb = set_table_dtypes(train_static_cb)
train_person_1 = set_table_dtypes(train_person_1)
train_credit_bureau_b_2 = set_table_dtypes(train_credit_bureau_b_2)

In [7]:
# Read CSV files for test data
test_static_0_0 = set_table_dtypes(test_static_0_0)
test_static_0_1 = set_table_dtypes(test_static_0_1)
test_static_0_2 = set_table_dtypes(test_static_0_2) # NOT NEEDED
test_static_cb = set_table_dtypes(test_static_cb)
test_person_1 = set_table_dtypes(test_person_1)
test_credit_bureau_b_2 = set_table_dtypes(test_credit_bureau_b_2)

# Merging

In [8]:
import pandas as pd

# Define the aggregation function
def aggregate_train_person_1_feats_1(df):
    agg_df = df.groupby("case_id").agg(
        mainoccupationinc_384A_max=pd.NamedAgg(column="mainoccupationinc_384A", aggfunc="max"),
        mainoccupationinc_384A_any_selfemployed=pd.NamedAgg(column="incometype_1044T", aggfunc=lambda x: (x == "SELFEMPLOYED").max())
    ).reset_index()
    return agg_df

# Apply the aggregation function
train_person_1_feats_1 = aggregate_train_person_1_feats_1(train_person_1)

# Filter and select operations
train_person_1_feats_2 = train_person_1.loc[train_person_1["num_group1"] == 0, ["case_id", "housetype_905L"]]
train_person_1_feats_2.rename(columns={"housetype_905L": "person_housetype"}, inplace=True)

# Define the aggregation function for another table
def aggregate_train_credit_bureau_b_2_feats(df):
    agg_df = df.groupby("case_id").agg(
        pmts_pmtsoverdue_635A_max=pd.NamedAgg(column="pmts_pmtsoverdue_635A", aggfunc="max"),
        pmts_dpdvalue_108P_over31=pd.NamedAgg(column="pmts_dpdvalue_108P", aggfunc=lambda x: (x > 31).max())
    ).reset_index()
    return agg_df

# Apply the aggregation function
train_credit_bureau_b_2_feats = aggregate_train_credit_bureau_b_2_feats(train_credit_bureau_b_2)

# Selecting columns based on their suffix
selected_static_cols = [col for col in train_static.columns if col[-1] in ("A", "M")]
print(selected_static_cols)

selected_static_cb_cols = [col for col in train_static_cb.columns if col[-1] in ("A", "M")]
print(selected_static_cb_cols)

# Joining tables together
data = train_basetable.merge(
    train_static[["case_id"] + selected_static_cols], how="left", on="case_id"
).merge(
    train_static_cb[["case_id"] + selected_static_cb_cols], how="left", on="case_id"
).merge(
    train_person_1_feats_1, how="left", on="case_id"
).merge(
    train_person_1_feats_2, how="left", on="case_id"
).merge(
    train_credit_bureau_b_2_feats, how="left", on="case_id"
)


#1m 51s


['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [9]:
import pandas as pd

# Define the aggregation function for test_person_1_feats_1
def aggregate_test_person_1_feats_1(df):
    agg_df = df.groupby("case_id").agg(
        mainoccupationinc_384A_max=pd.NamedAgg(column="mainoccupationinc_384A", aggfunc="max"),
        mainoccupationinc_384A_any_selfemployed=pd.NamedAgg(column="incometype_1044T", aggfunc=lambda x: (x == "SELFEMPLOYED").max())
    ).reset_index()
    return agg_df

# Apply the aggregation function
test_person_1_feats_1 = aggregate_test_person_1_feats_1(test_person_1)

# Filter and select operations for test_person_1_feats_2
test_person_1_feats_2 = test_person_1.loc[test_person_1["num_group1"] == 0, ["case_id", "housetype_905L"]]
test_person_1_feats_2.rename(columns={"housetype_905L": "person_housetype"}, inplace=True)

# Define the aggregation function for test_credit_bureau_b_2_feats
def aggregate_test_credit_bureau_b_2_feats(df):
    agg_df = df.groupby("case_id").agg(
        pmts_pmtsoverdue_635A_max=pd.NamedAgg(column="pmts_pmtsoverdue_635A", aggfunc="max"),
        pmts_dpdvalue_108P_over31=pd.NamedAgg(column="pmts_dpdvalue_108P", aggfunc=lambda x: (x > 31).max())
    ).reset_index()
    return agg_df

# Apply the aggregation function
test_credit_bureau_b_2_feats = aggregate_test_credit_bureau_b_2_feats(test_credit_bureau_b_2)

# Joining tables together
data_submission = test_basetable.merge(
    test_static[["case_id"] + selected_static_cols], how="left", on="case_id"
).merge(
    test_static_cb[["case_id"] + selected_static_cb_cols], how="left", on="case_id"
).merge(
    test_person_1_feats_1, how="left", on="case_id"
).merge(
    test_person_1_feats_2, how="left", on="case_id"
).merge(
    test_credit_bureau_b_2_feats, how="left", on="case_id"
)


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Shuffle the case_ids and split into train, validation, and test sets
case_ids = data["case_id"].unique()
case_ids = pd.Series(case_ids).sample(frac=1, random_state=1).to_numpy()
case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.6, random_state=1)
case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5, random_state=1)

# Extract columns for prediction
cols_pred = [col for col in data.columns if col[-1].isupper() and col[:-1].islower()]

print(cols_pred)

# Function to filter data and convert to pandas DataFrame
def from_polars_to_pandas(case_ids, data):
    filtered_data = data[data["case_id"].isin(case_ids)]
    return (
        filtered_data[["case_id", "WEEK_NUM", "target"]],
        filtered_data[cols_pred],
        filtered_data["target"]
    )

# Split data into train, validation, and test sets
base_train, X_train, y_train = from_polars_to_pandas(case_ids_train, data)
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid, data)
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test, data)

# Convert string columns to category
for df in [X_train, X_valid, X_test]:
    df = convert_strings(df)


['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [11]:
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {X_test.shape}")


Train: (915995, 48)
Valid: (305332, 48)
Test: (305332, 48)


Lightgmb model


In [12]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
)

c:\Users\Maevex\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.699919
[100]	valid_0's auc: 0.717512
[150]	valid_0's auc: 0.7247
[200]	valid_0's auc: 0.728744
[250]	valid_0's auc: 0.731757
[300]	valid_0's auc: 0.734638
[350]	valid_0's auc: 0.736952
[400]	valid_0's auc: 0.738423
[450]	valid_0's auc: 0.739968
[500]	valid_0's auc: 0.74133
[550]	valid_0's auc: 0.742478
[600]	valid_0's auc: 0.743322
[650]	valid_0's auc: 0.744115
[700]	valid_0's auc: 0.744701
[750]	valid_0's auc: 0.745422
Early stopping, best iteration is:
[765]	valid_0's auc: 0.745759


Nueral Network

In [13]:
import pandas as pd

# Assuming 'X_train' and 'X_valid' are pandas DataFrames
X_train = pd.get_dummies(X_train, drop_first=True)
X_valid = pd.get_dummies(X_valid, drop_first=True)

# Ensure alignment of features in training and validation set
X_train, X_valid = X_train.align(X_valid, join='inner', axis=1)  # align columns


In [14]:
from sklearn.preprocessing import LabelEncoder

# Label encoding for each categorical column
for column in X_train.select_dtypes(include=['object', 'category']).columns:
    le = LabelEncoder()
    X_train[column] = le.fit_transform(X_train[column].astype(str))
    X_valid[column] = le.transform(X_valid[column].astype(str))


In [15]:
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')


In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)


In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# Define the model architecture
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer
    layers.Dropout(0.2),  # Dropout for regularization
    layers.Dense(64, activation='relu'),  # Hidden layer
    layers.Dropout(0.2),  # Dropout for regularization
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model with appropriate loss function, optimizer and metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.05),
    loss='binary_crossentropy',  # Appropriate loss for binary classification
    metrics=['AUC']  # Metric as AUC, consistent with your LightGBM model
)

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=1000,  # Maximum number of epochs
    batch_size=32,  # Batch size for the training
    callbacks=[early_stopping],  # Callbacks for early stopping
    verbose=1
)


c:\Users\Maevex\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
28625/28625 ━━━━━━━━━━━━━━━━━━━━ 30s 1ms/step - AUC: 0.5038 - loss: 0.1432 - val_AUC: 0.5000 - val_loss: 0.1395
Epoch 2/1000
  121/28625 ━━━━━━━━━━━━━━━━━━━━ 23s 834us/step - AUC: 0.5030 - loss: 0.1551   

c:\Users\Maevex\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,loss,val_AUC,val_loss
  current = self.get_monitor_value(logs)


28625/28625 ━━━━━━━━━━━━━━━━━━━━ 28s 985us/step - AUC: 0.4991 - loss: 0.1406 - val_AUC: 0.5000 - val_loss: 0.1394
Epoch 3/1000
28625/28625 ━━━━━━━━━━━━━━━━━━━━ 29s 1ms/step - AUC: 0.4992 - loss: 0.1401 - val_AUC: 0.5000 - val_loss: 0.1391
Epoch 4/1000
28625/28625 ━━━━━━━━━━━━━━━━━━━━ 29s 997us/step - AUC: 0.4998 - loss: 0.1402 - val_AUC: 0.5000 - val_loss: 0.1401
Epoch 5/1000
28625/28625 ━━━━━━━━━━━━━━━━━━━━ 29s 998us/step - AUC: 0.4988 - loss: 0.1400 - val_AUC: 0.5000 - val_loss: 0.1396
Epoch 6/1000
28625/28625 ━━━━━━━━━━━━━━━━━━━━ 29s 1000us/step - AUC: 0.4976 - loss: 0.1402 - val_AUC: 0.5000 - val_loss: 0.1391
Epoch 7/1000
28625/28625 ━━━━━━━━━━━━━━━━━━━━ 28s 993us/step - AUC: 0.4997 - loss: 0.1394 - val_AUC: 0.5000 - val_loss: 0.1391
Epoch 8/1000
28625/28625 ━━━━━━━━━━━━━━━━━━━━ 28s 985us/step - AUC: 0.5013 - loss: 0.1391 - val_AUC: 0.5000 - val_loss: 0.1400
Epoch 9/1000
28017/28625 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step - AUC: 0.4977 - loss: 0.1398